In [61]:


import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")

In [62]:
customers = pd.read_csv('./assets/recommend_1.csv')
transactions = pd.read_csv('./assets/trx_data.csv')

In [63]:
print(customers.shape)
customers.head()
print(transactions.shape)
transactions.head()

(1000, 1)
(62483, 2)


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2


In [64]:
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split('|')])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

,customerId,0,1,2,3,4,5,6,7,8,9
0,0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,2.0,23.0,68.0,68.0,111.0,29.0,86.0,107.0,152.0


In [65]:
# example 2: organize a given table into a dataframe with customerId, single productId, and purchase count
pd.melt(transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,customerId,productId,purchase_count
0,0,20.0,1
1,1,2.0,2
2,1,23.0,1
3,1,29.0,1
4,1,68.0,2
5,1,86.0,1
6,1,107.0,1
7,1,111.0,1
8,1,152.0,1


In [66]:
s=time.time()

data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.13 minutes


In [67]:
print(data.shape)
data.head()

(133585, 3)


,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


In [68]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
    

In [69]:
data_dummy = create_data_dummy(data)

In [70]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
(df_matrix.shape)

(24429, 300)

In [72]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(24429, 300)


productId,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
customerId,,,,,,,,,,,,,,,,,,,,,
0,NaN,0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(133585, 3)


,customerId,productId,scaled_purchase_freq
9,9,0,0.133333
25,25,0,0.133333
32,33,0,0.133333
35,36,0,0.133333
43,44,0,0.133333


In [76]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
    

In [77]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(106868, 3) (26717, 3)


In [78]:
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

In [79]:
train_data

customerId,productId,purchase_count
9833,156,1
2298,216,1
13974,238,1
154,222,2
2194,57,1
4390,19,1
7352,228,3
9334,210,1
25878,13,1
7223,168,3


In [80]:
test_data

customerId,productId,purchase_count
8957,44,1
12814,149,1
11704,128,2
5529,295,1
116,61,2
26742,96,2
2826,156,1
27209,163,1
5664,63,1
14852,68,3


In [81]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [82]:
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [83]:
user_id = 'customerId'
item_id = 'productId'
target = 'purchase_count'
users_to_recommend = list(transactions[user_id])
n_rec = 10 # number of items to recommend
n_display = 30

In [84]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 106868 observations with 23268 users and 300 items.

Data prepared in: 0.132022s

106868 observations to process; with 300 unique items.

In [85]:
# Get recommendations for a list of users to recommend (from customers file)
# Printed below is head / top 30 rows for first 3 customers with 10 recommendations each

popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/62483 queries. users per second: 51461.5

recommendations finished on 2000/62483 queries. users per second: 88280.7

recommendations finished on 3000/62483 queries. users per second: 122724

recommendations finished on 4000/62483 queries. users per second: 148187

recommendations finished on 5000/62483 queries. users per second: 173871

...

recommendations finished on 59000/62483 queries. users per second: 373846

recommendations finished on 60000/62483 queries. users per second: 374782

recommendations finished on 61000/62483 queries. users per second: 377086

recommendations finished on 62000/62483 queries. users per second: 378707

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|     0      |    132    | 3.274193548387097  |  1   |
|     0      |    248    | 3.0681818181818183 |  2   |
|     0      |     34    | 3.0268199233716473 |  3   |
|     0      |     0     | 2.9668560606060606 |  4   |
|     0      |     37    | 2.964980544747082  |  5   |
|     0      |     3     | 2.8556263269639066 |  6   |
|     0      |     27    | 2.7642857142857142 |  7   |
|     0      |    110    | 2.6604938271604937 |  8   |
|     0      |     32    | 2.658291457286432  |  9   |
|     0      |     82    | 2.6331877729257642 |  10  |
|     1      |    132    | 3.274193548387097  |  1   |
|     1      |    248    | 3.0681818181818183 |  2   |
|     1      |     34    | 3.0268199233716473 |  3   |
|     1      |     0     | 2.9668560606060606 |  4   |
|     1      |     37    | 2.964980544747082  |  5   |
|     1   

In [86]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [87]:
ser_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to print the head / first few rows in a defined dataset

In [88]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23258 users and 300 items.

Data prepared in: 0.095012s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 159033

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     9     |  1.0  |  1   |
|    1553    |    246    |  1.0  |  2   |
|    1553    |    122    |  1.0  |  3   |
|    1553    |    249    |  1.0  |  4   |
|    1553    |    192    |  1.0  |  5   |
|    1553    |    141    |  1.0  |  6   |
|    1553    |     45    |  1.0  |  7   |
|    1553    |    209    |  1.0  |  8   |
|    1553    |     7     |  1.0  |  9   |
|    1553    |    110    |  1.0  |  10  |
|   20400    |     9     |  1.0  |  1   |
|   20400    |    246    |  1.0  |  2   |
|   20400    |    122    |  1.0  |  3   |
|   20400    |    249    |  1.0  |  4   |
|   20400    |    192    |  1.0  |  5   |
|   20400    |    141    |  1.0  |  6   |
|   20400    |     45    |  1.0  |  7   |
|   20400    |    209    |  1.0  |  8   |
|   20400    |     7     |  1.0  |  9   |
|   20400    |    110    |  1.0  |  10  |
|   19750    |     9     |  1.0  |

In [89]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23341 users and 300 items.

Data prepared in: 0.068846s

106868 observations to process; with 300 unique items.

recommendations finished on 1000/1000 queries. users per second: 155860

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7808219178082192 |  1   |
|    1553    |    230    |  0.328057553956834  |  2   |
|    1553    |    247    |  0.3229974160206719 |  3   |
|    1553    |    294    |  0.2644628099173551 |  4   |
|    1553    |    125    |  0.2637037037037032 |  5   |
|    1553    |    248    |  0.2553191489361702 |  6   |
|    1553    |    155    | 0.23846153846153836 |  7   |
|    1553    |    204    | 0.23655913978494605 |  8   |
|    1553    |    276    | 0.23461538461538461 |  9   |
|    1553    |     72    | 0.22950819672131148 |  10  |
|   20400    |    226    |  0.7808219178082192 |  1   |
|   20400    |    230    |  0.328057553956834  |  2   |
|   20400    |    247    |  0.3229974160206719 |  3   |
|   20400    |    294    |  0.2644628099173551 |  4   |
|   20400    |    125    |  0.2637037037037032 |

In [90]:
train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
132    3.274194
248    3.068182
34     3.026820
0      2.966856
37     2.964981
3      2.855626
27     2.764286
110    2.660494
32     2.658291
82     2.633188
226    2.628571
230    2.624060
10     2.583082
129    2.511236
173    2.490909
58     2.483146
83     2.448276
68     2.427350
41     2.425325
91     2.409091
Name: purchase_count, dtype: float64

In [91]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23268 users and 300 items.

Data prepared in: 0.069578s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.016433s

recommendations finished on 1000/1000 queries. users per second: 62762.8

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.09611833095550537  |  1   |
|    1553    |     5     | 0.07466095685958862  |  2   |
|    1553    |     17    |  0.0740822434425354  |  3   |
|    1553    |     1     | 0.07150241136550903  |  4   |
|    1553    |     35    | 0.05555305480957031  |  5   |
|    1553    |    162    |  0.048896324634552   |  6   |
|    1553    |    124    | 0.04777834415435791  |  7   |
|    1553    |     61    | 0.045536267757415774 |  8   |
|    1553    |     33    | 0.04547817707061767  |  9   |
|    1553    |    167    | 0.040196990966796874 |  10  |
|   20400    |    246    |  0.0479048490524292  |  1   |
|   20400    |    160    | 0.04294389486312866  |  2   |
|   20400    |    122    | 0.042781054973602295 |  3   |
|   20400    |    284    | 0.04174298048019409  |  4   |
|   20400    |     6     | 0.04

In [92]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23258 users and 300 items.

Data prepared in: 0.098008s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.050811s

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.10151444375514984  |  1   |
|    1553    |     5     | 0.07988397777080536  |  2   |
|    1553    |     35    |  0.0793403834104538  |  3   |
|    1553    |     17    | 0.06881895661354065  |  4   |
|    1553    |     1     | 0.060543134808540344 |  5   |
|    1553    |     8     | 0.054424017667770386 |  6   |
|    1553    |     11    | 0.050963833928108215 |  7   |
|    1553    |     61    | 0.049317553639411926 |  8   |
|    1553    |     9     | 0.04885376989841461  |  9   |
|    1553    |     36    | 0.046601518988609314 |  10  |
|   20400    |    113    | 0.051524579524993896 |  1   |
|   20400    |     85    | 0.04352247714996338  |  2   |
|   20400    |     27    |  0.0431901216506958  |  3   |
|   20400    |    224    | 0.04251587390899658  |  4   |
|   20400    |     54    |  0.0

recommendations finished on 1000/1000 queries. users per second: 105641

In [93]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23341 users and 300 items.

Data prepared in: 0.080926s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 3.553ms                        | 100        |

+------------+-----------+-----------------------+------+
| customerId | productId |         score         | rank |
+------------+-----------+-----------------------+------+
|    1553    |    118    |          0.0          |  1   |
|    1553    |     33    |          0.0          |  2   |
|    1553    |     16    |          0.0          |  3   |
|    1553    |     80    |          0.0          |  4   |
|    1553    |     32    |          0.0          |  5   |
|    1553    |    262    |          0.0          |  6   |
|    1553    |     51    |          0.0          |  7   |
|    1553    |    136    |          0.0          |  8   |
|    1553    |    106    |          0.0          |  9   |
|    1553    |    287    |          0.0          |  10  |
|   20400    |     2     | 0.0035287928581237793 |  1   |
|   20400    |     1     |  0.003511742353439331 |  2   |
|   20400    |     8     | 0.0023563170433044435 |  3   |
|   20400    |     0     |  0.002153496742248535 |  4   |
|   20400    |

+--------------------------------+------------+

Setting up lookup tables.

Processing data in one pass using dense lookup tables.

In [94]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23268 users and 300 items.

Data prepared in: 0.06824s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

| 5.71ms                              | 0                | 0               |

+------------+-----------+--------------------+------+
| customerId | productId |       score        | rank |
+------------+-----------+--------------------+------+
|    1553    |    132    | 3.2741935483870965 |  1   |
|    1553    |    248    | 3.064912467653102  |  2   |
|    1553    |     34    | 3.0094499139493456 |  3   |
|    1553    |     0     | 2.966856060606068  |  4   |
|    1553    |     37    | 2.9649805447470827 |  5   |
|    1553    |     3     | 2.8550414861893723 |  6   |
|    1553    |     27    | 2.7642857142857147 |  7   |
|    1553    |    110    | 2.6595805409513877 |  8   |
|    1553    |     32    | 2.6582914572864316 |  9   |
|    1553    |     82    | 2.630356754954727  |  10  |
|   20400    |    132    | 3.265402518933819  |  1   |
|   20400    |    248    | 3.0681818181818192 |  2   |
|   20400    |     34    | 3.0268199233716477 |  3   |
|   20400    |     0     | 2.966856060606068  |  4   |
|   20400    |     37    | 2.9638395330321474 |  5   |
|   20400 

| 23.466ms                            | 100              | 300             |

+-------------------------------------+------------------+-----------------+

In [95]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 106868 observations with 23258 users and 300 items.

Data prepared in: 0.063078s

...

| 6.812ms                             | 0                | 0               |

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|    1553    |     9     |  0.0  |  1   |
|    1553    |    246    |  0.0  |  2   |
|    1553    |    122    |  0.0  |  3   |
|    1553    |    249    |  0.0  |  4   |
|    1553    |    192    |  0.0  |  5   |
|    1553    |    141    |  0.0  |  6   |
|    1553    |     45    |  0.0  |  7   |
|    1553    |    209    |  0.0  |  8   |
|    1553    |     7     |  0.0  |  9   |
|    1553    |    110    |  0.0  |  10  |
|   20400    |     9     |  0.0  |  1   |
|   20400    |    246    |  0.0  |  2   |
|   20400    |    122    |  0.0  |  3   |
|   20400    |    249    |  0.0  |  4   |
|   20400    |    192    |  0.0  |  5   |
|   20400    |    141    |  0.0  |  6   |
|   20400    |     45    |  0.0  |  7   |
|   20400    |    209    |  0.0  |  8   |
|   20400    |     7     |  0.0  |  9   |
|   20400    |    110    |  0.0  |  10  |
|   19750    |     9     |  0.0  |

| 19.265ms                            | 100              | 300             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

In [96]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 106868 observations with 23341 users and 300 items.

Data prepared in: 0.068243s

Training model from provided data.

Gathering per-item and per-user statistics.

...

Processing data in one pass using dense lookup tables.

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|    1553    |    226    |  0.7805972082974157 |  1   |
|    1553    |    230    | 0.32770691175254973 |  2   |
|    1553    |    247    | 0.32299741602067183 |  3   |
|    1553    |    294    |  0.2644628099173552 |  4   |
|    1553    |    125    | 0.26341907960397226 |  5   |
|    1553    |    248    |  0.2553191489361702 |  6   |
|    1553    |    155    | 0.23846153846153845 |  7   |
|    1553    |    204    | 0.23655913978494625 |  8   |
|    1553    |    276    | 0.23424283174368057 |  9   |
|    1553    |     72    |   0.22922379911923  |  10  |
|   20400    |    226    |  0.7808096011044225 |  1   |
|   20400    |    230    |  0.3280463125208298 |  2   |
|   20400    |    247    | 0.32299470400933455 |  3   |
|   20400    |    294    |  0.2644442132681853 |  4   |
|   20400    |    125    |  0.2636924873016499 |

+-------------------------------------+------------------+-----------------+

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

+-------------------------------------+------------------+-----------------+

In [97]:
models_w_counts = [popularity_model, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [98]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/13878 queries. users per second: 75568.7

recommendations finished on 2000/13878 queries. users per second: 101518

recommendations finished on 3000/13878 queries. users per second: 111940

recommendations finished on 4000/13878 queries. users per second: 114025

...

recommendations finished on 10000/13878 queries. users per second: 151909

recommendations finished on 11000/13878 queries. users per second: 154388

recommendations finished on 12000/13878 queries. users per second: 155966

recommendations finished on 13000/13878 queries. users per second: 156912


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0009367343997694177 | 0.0006148820675409522 |
|   2    |  0.000828649661334486 |  0.001052024787433349 |
|   3    | 0.0022097324302252966 |  0.003532501812247871 |
|   4    |  0.005548349906326566 |  0.012047177317844257 |
|   5    | 0.0059518662631503514 |  0.01607101772300761  |
|   6    |  0.006389008983042724 |  0.02053551789113941  |
|   7    |  0.00576451938319643  |  0.021691509903162598 |
|   8    |  0.005503314598645335 |  0.023780337017304237 |
|   9    |  0.005364205537141164 |  0.026036477259825788 |
|   10   |  0.005166450497189818 |  0.02787131577322029  |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0404484054617749

Per User RMSE (best)
+------------+------------------------+-----

In [99]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy


recommendations finished on 1000/13875 queries. users per second: 87085.3

recommendations finished on 2000/13875 queries. users per second: 114345

recommendations finished on 3000/13875 queries. users per second: 122961

recommendations finished on 4000/13875 queries. users per second: 132882

...

recommendations finished on 10000/13875 queries. users per second: 156421

recommendations finished on 11000/13875 queries. users per second: 157778

recommendations finished on 12000/13875 queries. users per second: 157551

recommendations finished on 13000/13875 queries. users per second: 155409


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0032432432432432474 | 0.0015414999414999396 |
|   2    |  0.01127927927927927  |  0.012499654979654985 |
|   3    |  0.00912912912912908  |  0.015105946985946988 |
|   4    |  0.00963963963963963  |  0.021478137087548872 |
|   5    |  0.008936936936937033 |  0.024681221490633204 |
|   6    |  0.008276276276276317 |  0.02753916514857687  |
|   7    |  0.007660231660231662 |  0.02979550148491322  |
|   8    |  0.007531531531531532 |  0.03351838280779453  |
|   9    |  0.007295295295295389 |  0.03672759201700365  |
|   10   |  0.008050450450450365 |  0.04510937984400901  |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |

In [100]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/13852 queries. users per second: 89525.5

recommendations finished on 2000/13852 queries. users per second: 115174

recommendations finished on 3000/13852 queries. users per second: 127378

recommendations finished on 4000/13852 queries. users per second: 135350

...

recommendations finished on 10000/13852 queries. users per second: 142493

recommendations finished on 11000/13852 queries. users per second: 143308

recommendations finished on 12000/13852 queries. users per second: 143426

recommendations finished on 13000/13852 queries. users per second: 143050


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0025267109442679795 | 0.0015360511805068554 |
|   2    |  0.002274039849841182 | 0.0027524820208187184 |
|   3    | 0.0022379439792087744 |  0.003899072820528211 |
|   4    | 0.0024545192030031773 |  0.00560437132412299  |
|   5    |  0.002425642506497252 |  0.007066340027284297 |
|   6    | 0.0021296563673115806 | 0.0073781682430253005 |
|   7    |  0.002062621178994262 |  0.008292555560193125 |
|   8    | 0.0019491770141495813 |  0.00887269641430162  |
|   9    | 0.0020614752783392693 |  0.010233281557012124 |
|   10   | 0.0027144094715564715 |  0.014550691454843617 |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.13392326087384093

Per User RMSE (best)
+------------+------------------------+----

In [101]:
users_to_recommend = list(customers[user_id])

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', 
                                            similarity_type='cosine')

recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 133585 observations with 24429 users and 300 items.

Data prepared in: 0.095617s

...

Processing data in one pass using dense lookup tables.

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|    1553    |     2     | 0.10592188835144042  |  1   |
|    1553    |     1     | 0.10348175764083863  |  2   |
|    1553    |     35    | 0.07407552003860474  |  3   |
|    1553    |     33    |  0.0668614387512207  |  4   |
|    1553    |     61    | 0.06512556076049805  |  5   |
|    1553    |     15    | 0.06476415395736694  |  6   |
|    1553    |     13    | 0.05799754858016968  |  7   |
|    1553    |     20    | 0.05683724880218506  |  8   |
|    1553    |     29    | 0.05509240627288818  |  9   |
|    1553    |     11    | 0.05467898845672607  |  10  |
|   20400    |     26    | 0.05812269449234009  |  1   |
|   20400    |     6     | 0.05361741781234741  |  2   |
|   20400    |    113    | 0.05312788486480713  |  3   |
|   20400    |     1     | 0.05210459232330322  |  4   |
|   20400    |     15    | 0.04

+-------------------------------------+------------------+-----------------+

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

+-------------------------------------+------------------+-----------------+

In [102]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(10000, 4)


,customerId,productId,score,rank
0,1553,2,0.105922,1
1,1553,1,0.103482,2
2,1553,35,0.074076,3
3,1553,33,0.066861,4
4,1553,61,0.065126,5


In [103]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates().sort_values('customerId').set_index('customerId')

In [104]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('./assets/outputfin.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [105]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/1000 queries. users per second: 169664

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(1000, 1)


,recommendedProducts
customerId,
4,226|230|247|294|125|248|155|204|276|72
11,226|230|247|294|125|248|155|204|276|72
12,226|230|247|294|125|248|155|204|276|72
16,226|230|247|294|125|248|155|204|276|72
21,226|230|247|294|125|248|155|204|276|72


In [106]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [107]:
customer_recomendation(4)

recommendedProducts    226|230|247|294|125|248|155|204|276|72
Name: 4, dtype: object

In [108]:
customer_recomendation(21)

recommendedProducts    226|230|247|294|125|248|155|204|276|72
Name: 21, dtype: object

In [109]:
customer_recomendation(334)

Customer not found.


334